In [1]:
import pandas as pd
import random
import numpy as np
#!pip install nltk
import nltk
import string
#nltk.download()
#!pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
#nltk.download('wordnet')
import unicodedata
#!pip install inflect -- user
import inflect

In [2]:
#Loading and Cleaning Data

#load in the csv contatining the data
df_original = pd.read_csv('wiki_movie_plots_deduped.csv')

In [3]:
#drop all rows in the dataframe that do not contain horror(1167) in genre catergory
df_horror = df_original.drop(df_original[df_original['Genre'] != 'horror'].index,inplace=False)
#drop columns we wont need
df_horror.drop(['Release Year','Origin/Ethnicity','Cast', 'Wiki Page','Director','Release Year'],axis=1, 
               inplace = True)

#reduce horror to 1000 rows
df_horror = df_horror.sample(frac=1)
df_horror = df_horror.reset_index(drop=True)
df_horror_train = df_horror[0:1000]
df_horror_test = df_horror[1000:len(df_horror)]

In [4]:
#drop all rows in dataframe that do not contain comedy(4379)
df_comedy = df_original.drop(df_original[df_original['Genre'] != 'comedy'].index, inplace = False)
df_comedy.drop(['Release Year','Origin/Ethnicity','Cast', 'Wiki Page','Director','Release Year'],axis=1, 
               inplace = True)
#reduce horror to 1000 rows
df_comedy = df_comedy.sample(frac=1)
df_comedy = df_comedy.reset_index(drop=True)
df_comedy_train = df_comedy[0:1000]
df_comedy_test = df_comedy[1000:len(df_comedy)]

In [5]:
#merge the comedy and horror dataframes into one
df_HandC = pd.concat([df_horror_test,df_comedy_test])

#Pre punctuation parsing test
#df_HandC.iloc[1002]['Plot']

In [6]:
#Function to get rid of punctuation
def normalizeText(row):
    #create a list of punctuations we wish to delete
    punctuations = string.punctuation
    
    #create a list of stopwords in English
    stopWords = nltk.corpus.stopwords.words('english')
    
    #change all letters to lower case
    row=row.lower()  
    
    #loop through each word and get rid of punctuation and \r\n that came with data
    # also do the lemmatizer step where played is converted to 
    # also remove the stop words such as a/the/of/...
    for x in row: 
        lemmatizer = WordNetLemmatizer()
        lemmatizer.lemmatize(x)
        
        if x in punctuations: 
            row = row.replace(x, "")
        if x in stopWords:
            row = row.replace(x,"")
        row = row.replace('\r\n', '')
    
    return row
#run the column through the normalizing function
df_HandC["Plot"] = df_HandC["Plot"].apply(normalizeText) 

In [7]:
# Remove accented characters.
# Param: entire string will be converted, punctuation accepted
def removeAccents(string):
    return unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [8]:
# Replace numbers with word representation.
# NOTE: run before removing punctuation, result will have punctuation
# Param: single num, no punctuation or spaces within the string
def numToWord(num):
    try:
        float(num)
        return inflect.engine().number_to_words(num)
    except ValueError:
        return

In [9]:
#Post Punctuation Parsing test
#df_HandC.iloc[1002]['Plot']

In [10]:
##Prabesh being stupid. might be helpful?

#vectorizer = TfidfVectorizer()
#tfidf = vectorizer.fit_transform(df_HandC['Plot'])
#print similarity matrix
#print((tfidf*tfidf.T).A)

# Make list from df for "Plot" column
# Param: the df to use
from itertools import chain
def dfToList(df):
    lst = []
    for i, rows in df.iterrows():
        l = [rows.Plot]
        lst.append(l)
    return list(chain.from_iterable(lst))

# tf-idf vectorisation
# Param: all plots from the df as a list (ie [plot1, plot2,...])
def tfidfVec(plots):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(plots)
    feature_names = vectorizer.get_feature_names()
    dense = tfidf.todense()
    df_tfidf = pd.DataFrame(dense.tolist(), columns=feature_names)
    return df_tfidf

In [11]:
# test showing how it works on first 5 elements of df_HandC
tfidfVec(dfToList(df_HandC.head()))


,10,10h,11,1980,bbe,bck,be,bece,becue,been,...,wr,wren,wrk,wrlel,wrng,wun,zbe,zben,zucc,zurk
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.013328,0.022127,0.000000,0.026656,0.013328,...,0.013328,0.000000,0.000000,0.016519,0.000000,0.000000,0.214753,0.016519,0.000000,0.000000
2,0.028884,0.014442,0.014442,0.014442,0.014442,0.046608,0.048361,0.011652,0.011652,0.023304,...,0.000000,0.014442,0.014442,0.000000,0.014442,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033223,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.041179,0.000000,0.000000,0.041179,0.041179
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031888,0.000000,0.000000,0.000000,...,0.038415,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
